<center><h1>Assignment 3</h1>
<h1> Distributional Semantics and POS Tagging </h1> </center>
<hr>
<br><b><u>Submitted by:</u><br>
Name: Adithya Avvaru<br>
Roll No: 20162116
</b>

<h1> Latent Symantic Analysis </h1>

In [1]:
import io
from pprint import pprint
from time import time
def get_unigrams(file_name):
    unigrams = {}
    with io.open(file_name, encoding='utf8', errors='ignore') as f:
        for line in f:
            tokens = line.strip().split()
            
            # Added to extract only Verbs
            # verbs = nltk.pos_tag(tokens)
            # tokens = set([x for x,y in verbs if "VB" in y]) 
            
            for token in tokens:
                token = token.lower()
                try:
                    unigrams[token]
                except:
                    unigrams[token] = 0
                unigrams[token] += 1
                
    return unigrams

def index_unigrams(unigrams):
    new_unigrams = {}
    reverse_unigrams = {}
    for index, unigram in enumerate(unigrams):
        new_unigrams[unigram] = index
        reverse_unigrams[index] = unigram
    return new_unigrams, reverse_unigrams
            

In [4]:
file_name = "sample_corpus.txt"
unigrams = get_unigrams(file_name)
iunigrams,runigrams = index_unigrams(unigrams)
unigrams = sorted(unigrams.items(), key = lambda x: x[1], reverse = True )

#pprint(unigrams) # Figure out non-stop words
dimensions = [x[0] for x in unigrams[100:3100]]
idimensions = {x: index for index, x in enumerate(dimensions)}

In [5]:
import numpy
import nltk
from nltk import word_tokenize

def populate_cmatrix(file_name,iunigrams, dimensions, window, leftonly, rightonly):
    e = 0
    s = 0
    cmatrix = numpy.memmap("lsa.cmatrix", dtype='float32', mode='w+', shape=(len(unigrams),len(dimensions)))
    with open(file_name, encoding='utf-8', errors='ignore') as f:
        for index, line in enumerate(f):             
            tokens = line.strip().split()
            for indexj, token in enumerate(tokens):
                token = token.lower()
                lcontext = rcontext = ""
                if leftonly :
                    lcontext = tokens[indexj - window:indexj]
                else:
                    lcontext = []
                if rightonly:
                    rcontext = tokens[indexj + 1:index + window]
                else:
                    rcontext = []
                context = [tok.lower() for tok in lcontext + rcontext]
                
                #verbs = nltk.pos_tag(context)
                #context = set([x for x,y in verbs if "VB" in y]) 
                
                try:
                    unigram_index = iunigrams[token]                    
                    for d in context:
                        #print(nltk.pos_tag([d]))
                        
                        if d in dimensions:
                            j = dimensions[d]
                            cmatrix[unigram_index][j] += 1 
                            s += 1
                except:
                    e += 1
    #print(e,s)
    return cmatrix

In [6]:
from scipy.spatial.distance import *
from numpy import linalg as LA
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [7]:
def getDistance(twod_cmatrix):
    words = ["boy","sunday","eat","good","slowly","100"]
    for w1 in words:
        distance = {}
        for w2 in idimensions:
            if w1 == w2:
                continue
            id1 = iunigrams[w1]
            id2 = iunigrams[w2]
            v1, v2 = twod_cmatrix[id1], twod_cmatrix[id2]
            if np.linalg.norm(v2) == 0:
                continue
            distance[w2] = cosine(v1,v2)
        sortedDistance = sorted(distance.items(), key = lambda x : x[1], reverse = False)
        
        temp = dict((x, y) for x, y in sortedDistance[:10])  # For top 10 elements
        
        print(w1)
        print(list(temp.keys()))

In [8]:
def run(windowSize=2, noComp=10, leftOnly=True, rightOnly=True):
    s = time()
    cmatrix = populate_cmatrix(file_name, iunigrams, idimensions, window = windowSize, \
                               leftonly = leftOnly, rightonly = rightOnly)
    svd = TruncatedSVD(n_components = noComp, random_state=42)
    svd.fit(cmatrix)
    twod_cmatrix = svd.transform(cmatrix)
    
    print("Window :",windowSize,"; No components :", noComp, "; Left only :", leftOnly, "; Right only :", rightOnly)
    print("-----------------------------------------------------------------------")
    print("Time taken is ---- ", (time()-s))
    getDistance(twod_cmatrix)

In [9]:
run(windowSize=2, noComp=10)

Window : 2 ; No components : 10 ; Left only : True ; Right only : True
-----------------------------------------------------------------------
Time taken is ----  59.825199842453
boy
['supplied', 'miles', 'adobe', 'attend', 'speaker', 'forced', 'leisure', 'nursing', 'birds', 'minutes']
sunday
['saturday', 'director', 'morning', 'routes', 'ford', '17th', 'seminar', 'evening', 'supported', '1st']
eat
['saying', 'think', 'me', "'ve", 'sit', 'am', 'heavy', 'sorry', "'re", "'ll"]
good
['him', 'thanks', 'always', 'though', 'went', 'see', 'take', 'nor', 'often', 'god']
slowly
['understanding', 'developed', 'brand', 'positive', 'meant', 'parents', 'helpful', 'illness', 'residents', 'clear']
100
['white', '90', 'column', 'black', '=', 'hotels', 'cm', 'pp', '1000', 'mm']


In [ ]:
run(windowSize=2, noComp=50)

In [ ]:
run(windowSize=2, noComp=100)

In [ ]:
run(windowSize=2, noComp=200)

In [ ]:
run(windowSize=5, noComp=10) 

In [ ]:
run(windowSize=5, noComp=50) 

In [ ]:
run(windowSize=5, noComp=100) 

In [ ]:
run(windowSize=5, noComp=200) 

In [ ]:
run(windowSize=10, noComp=10) 

In [ ]:
run(windowSize=10, noComp=50) 

In [ ]:
run(windowSize=10, noComp=100) 

In [ ]:
run(windowSize=10, noComp=200) 

In [ ]:
run(windowSize=2, noComp=10, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=2, noComp=50, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=2, noComp=100, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=2, noComp=200, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=5, noComp=10, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=5, noComp=50, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=5, noComp=100, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=5, noComp=200, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=10, noComp=10, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=10, noComp=50, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=10, noComp=100, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=10, noComp=200, leftOnly=True, rightOnly=False)

In [ ]:
run(windowSize=2, noComp=10, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=2, noComp=50, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=2, noComp=100, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=2, noComp=200, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=5, noComp=10, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=5, noComp=50, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=5, noComp=100, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=5, noComp=200, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=10, noComp=10, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=10, noComp=50, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=10, noComp=100, leftOnly=False, rightOnly=True)

In [ ]:
run(windowSize=10, noComp=200, leftOnly=False, rightOnly=True)

<h1> KMeans Clustering </h1>

In [ ]:
# K-means Clustering
from sklearn.cluster import KMeans

cmatrix = populate_cmatrix(file_name, iunigrams, idimensions, window = 5, leftonly=True, rightonly=True)
svd = TruncatedSVD(n_components = 100, random_state=42)
svd.fit(cmatrix)
cmatrix = svd.transform(cmatrix)

kmeans = KMeans(n_clusters=100, random_state=0).fit(cmatrix)
i=0
wordTolabel={}
for word in dimensions:
    wordTolabel[word] = kmeans.labels_[i]
    i=i+1
sortedByLabel = sorted(wordTolabel.items(), key = lambda x : x[1], reverse = False)
print(sortedByLabel)

<b>OBSERVATIONS:</b>
*************
The words didnt group into clusters of different POS tags. Majority of words are gouped into cluster 0. When number of clusters are increased, we find a different clustering scheme where words are distributed among other clusters also.
The above is the experiment with n_clusters = 100